# Introduction: Business Problem

An established America coffee shop company, “Coffee on the go”, plans to open a number of takeaway shops (no-sitting) in London. Given their takeaway business model, the company needs to determine where the new outlets should be optimally located within the different boroughs in Central London, within Zone 1 (the area served by the underground system in London is defined by zones and zone 1 is the most central).

The main variable taken into account to pinpoint the ideal locations is to identify the areas in Central London with the highest number of potential customers “street traffic” (people walking in a given area). From a previous consumer survey, we know that customers of takeaway coffee shops usually purchase a coffee in the morning on their way to the workplace. Since the majority of workers in central London commute to their workplace by underground, takeaway coffee shops should be located ideally close or within minimum walking distance to underground stations. 

The goal is to locate those shops in such a way that all the city underground stations are within minimal walking distance. Since there are lots of coffee shops in central London, we will try to detect locations that are not already crowded with existing outlets. We are also particularly interested in areas with no coffee shops in vicinity. We would also prefer locations close to underground stations that have a high “street traffic”, assuming that first two conditions are met.
We implement a K-Median model to get the optimal location of future shops. We will use this technique to generate a few most promising neighbourhood locations based on the above criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.


***

# Data

We decided to use a grid of locations centred around underground stations to define our potential locations. Based on the definition of our problem, factors that will be appraised to take our decision are:
* Identification of underground stations locations in central London.
* Estimate the number of commuters exiting each station on average per day.
* Number of and distance to existing coffee shops in the stations neighbourhood.

The Following data sources will be needed to extract/generate the required information:
* The list of tube stations and their exact locations are obtained from Transport for London (TfL) web-site (https://tfl.gov.uk/info-for/open-data-users).
* The information about numbers of passenger exiting each London Underground station (or number of exits) are obtained as well from Tfl; exits are defined as number of passengers passing gates or ticket barriers going from the platforms to the street.
* The number of coffee shops and their location in every underground station proximity will be obtained using the Foursquare API
* Coordinate of central London will be obtained using standard geocoding library functions. 

## Additional data insight
### Stations geo-locations
It can be surprisingly hard to find a nicely structured dataset of stations and geo locations. Luckily some TfL libraries had some CSVs buried in it; otherwise the following web-sites provide and alternative source of structured data on stations geo location:
* https://www.doogal.co.uk/london_stations.php
* https://commons.wikimedia.org/wiki/London_Underground_geographic_maps/CSV
    
### Passenger counts data
On the Transport for London (TfL) web-site https://tfl.gov.uk/info-for/open-data-users, under the Network statistics tab is possible to access passenger counts data. TfL collects information about passenger numbers entering and exiting London Underground stations, largely based on the Underground ticketing system gate data. Counts data is obtained during the autumn of each year and does not necessarily reflect whole-year annual demand. The data is adjusted to remove the effect of abnormal circumstances that may affect demand such as industrial action. We use data collected by TfL based on survey data up to 2017 and reconciled to Autumn 2017 counts. The data provides the number of exits for each underground station mapped by the survey; the data number of exits are reported by Time Period, namely: {Early, AM peak, Midday, PM Peak, Evening, Late, Total day}. Exits are defined as number of passengers passing gates or ticket barriers going from the platforms to the street. The exits number by time period for each station are provided alongside the unique station number, the Borough in which the station is located and the station name itself.



***


# The London Tube

I found it suprisingly hard to find a nicely structured dataset of stations and connections between them. Luckily this library had some CSVs buried in it with just what I was looking for. So i yanked them out and opened them in pandas.
https://www.doogal.co.uk/london_stations.php
https://commons.wikimedia.org/wiki/London_Underground_geographic_maps/CSV
crowding.data.tfl.gov.uk

In [1]:
# The code was removed by Watson Studio for sharing.

Total number of tube stations locations  266


,ID,Latitude,Longitude,Station Name,Station Number,Zone,total_lines,rail
0,1,51.5028,-0.2801,Acton Town,500.0,3.0,2,0
1,2,51.5143,-0.0755,Aldgate,502.0,1.0,2,0
2,3,51.5154,-0.0726,Aldgate East,503.0,1.0,2,0
4,5,51.5407,-0.2997,Alperton,505.0,4.0,1,0
5,7,51.5322,-0.1058,Angel,507.0,1.0,1,0
6,8,51.5653,-0.1353,Archway,508.0,2.5,1,0
7,9,51.6164,-0.1331,Arnos Grove,509.0,4.0,1,0
8,10,51.5586,-0.1059,Arsenal,510.0,2.0,1,0
9,11,51.5226,-0.1571,Baker Street,511.0,1.0,5,0
10,12,51.4431,-0.1525,Balham,512.0,3.0,1,1


On the Trasport for London (TfL) web-site https://tfl.gov.uk/info-for/open-data-users, under the Network statistics tab is possible to access passenger counts data. TfL collects information about passenger numbers entering and exiting London Underground stations, largely based on the Underground ticketing system gate data. Counts data is obtained during the autumn of each year and does not necessarily reflect whole-year annual demand. The data is adjusted to remove the effect of abnormal circumstances that may affect demand such as industrial action.

We use data collected by TfL based on survey data up to 2017 and reconciled to Autumn 2017 counts. The data provides the number of exits for each underground station mapped by the survey; the data number of exits are reported by Time Period, namely: 
{Early, AM peak, Midday, PM Peak, Evening, Late, Total day}. Exits are defined as number of passengers passing gates or ticket barriers going from the platforms to the street. The exits number by time period for each station are provided alongside the unique station number, the Borough in which the station is located and the station name itself.    
     
   


In [4]:
body = client_de32eb85ef4840c1b273bc5319d51693.get_object(Bucket='capstoneprojectapplieddatascience-donotdelete-pr-frb4mhxgtaehr5',Key='Total exits by borough-time of day.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

peakExit = pd.read_csv(body)
peakExit.columns = ['Borough','Station Number','Station Name','Time Period','Number exiting']

ExitPerStation = peakExit.loc[~peakExit['Station Number'].isin(['NaN'])]
ExitPerStation.reset_index(drop=True, inplace=True)
ExitPerStation.head(58)

,Borough,Station Number,Station Name,Time Period,Number exiting
0,Barking,514.0,Barking,Early,1139
1,Barking,514.0,Barking,AM peak,5990
2,Barking,514.0,Barking,Midday,6226
3,Barking,514.0,Barking,PM Peak,11325
4,Barking,514.0,Barking,Evening,4925
5,Barking,514.0,Barking,Late,1903
6,Barking,514.0,Barking,Total day,31507
7,Barking,518.0,Becontree,Early,89
8,Barking,518.0,Becontree,AM peak,461
9,Barking,518.0,Becontree,Midday,1253


In [5]:
# This table aggregates the number of exits per day by Borough

TotalForBorough = peakExit.loc[peakExit['Station Number'].isin(['NaN'])]
TotalForBorough.reset_index(drop=True, inplace=True)
TotalForBorough = TotalForBorough.drop(columns=['Station Number','Station Name'], axis = 1)
TotalForBorough = TotalForBorough.loc[TotalForBorough['Time Period'].isin([' Total day'])]
TotalForBorough.reset_index(drop=True, inplace=True)
print('Total number of Borough on survey', len(TotalForBorough))
TotalForBorough.sort_values('Number exiting', ascending=False).head(10)

Total number of Borough on survey 32


,Borough,Time Period,Number exiting
31,Westminster,Total day,1029767
4,Camden,Total day,513541
5,City of London,Total day,425289
20,Lambeth,Total day,306749
27,Southwark,Total day,259328
28,Tower Hamlets,Total day,251313
23,Newham,Total day,248361
19,Kensington & Chelsea,Total day,233535
18,Islington,Total day,225838
11,Hammersmith & Fulham,Total day,183301


In [6]:
stationsData = stations.merge(ExitPerStation, left_on='Station Number', right_on='Station Number',suffixes=('_left', '_right'))
stationsDataZone_1 = stationsData.loc[stationsData['Zone']==1]
# reset index, because we droped rows
stationsDataZone_1.reset_index(drop=True, inplace=True)
stationsDataZone_1_Total = stationsDataZone_1.loc[stationsDataZone_1['Time Period'].isin([' Total day'])]
stationsDataZone_1_Total.reset_index(drop=True, inplace=True)
stationsDataZone_1_Total.rename(columns={'Station Name_left': 'Station Name'}, inplace=True)
stationsDataZone_1_Total.drop(columns=['total_lines','rail','Station Name_right'], axis = 1,inplace=True)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
print('Total number of Stations on survey', len(stationsDataZone_1_Total))
stationsDataZone_1_Total.sort_values('Number exiting', ascending=False).head(200)

Total number of Stations on survey 59


,ID,Latitude,Longitude,Station Name,Station Number,Zone,Borough,Time Period,Number exiting
28,145,51.5308,-0.1238,King's Cross St. Pancras,625.0,1.0,Camden,Total day,147949
56,279,51.5036,-0.1143,Waterloo,747.0,1.0,Lambeth,Total day,147683
40,192,51.5150,-0.1415,Oxford Circus,669.0,1.0,Westminster,Total day,138502
4,13,51.5133,-0.0886,Bank / Monument,513.0,1.0,City of London,Total day,136749
5,166,51.5108,-0.0863,Bank / Monument,513.0,1.0,City of London,Total day,136749
54,273,51.4965,-0.1447,Victoria,741.0,1.0,Westminster,Total day,127176
33,156,51.5178,-0.0823,Liverpool Street,634.0,1.0,City of London,Total day,113087
34,157,51.5052,-0.0864,London Bridge,635.0,1.0,Southwark,Total day,108319
41,193,51.5154,-0.1755,Paddington,670.0,1.0,Westminster,Total day,81691
24,107,51.5067,-0.1428,Green Park,590.0,1.0,Westminster,Total day,69611


# Explore the neighborhoods around tube stations in London

In [8]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# convert an address into latitude and longitude values
address = 'London'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [9]:
# create map of London using latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('Folium installed')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [24]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=13)
# add markers to map
for lat, lng, borough, neighborhood in zip(stationsDataZone_1_Total['Latitude'], stationsDataZone_1_Total['Longitude'], stationsDataZone_1_Total['Borough'], stationsDataZone_1_Total['Station Name']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)    
map_london

In [26]:
# Use Foursquare API 
CLIENT_ID = 'REMOVED'# your Foursquare ID
CLIENT_SECRET = 'REMOVED'# # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


Search for a specific venue category

In [27]:
search_query = 'Coffee'
radius = 500
LIMIT = 100
print(search_query + ' .... OK!')

Coffee .... OK!


In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [29]:
import requests # library to handle requests
from pandas.io.json import json_normalize

In [30]:
#column_names = ['name','categories','address','cc','city','country','crossStreet','distance','formattedAddress','labeledLatLngs','lat','lng','neighborhood','postalCode','state','id']
#CoffeeShop = pd.DataFrame(columns=column_names)
CoffeeShop = []
for lat, lng, borough, station, exits in zip(stationsDataZone_1_Total['Latitude'], stationsDataZone_1_Total['Longitude'], stationsDataZone_1_Total['Borough'], stationsDataZone_1_Total['Station Name'],stationsDataZone_1_Total['Number exiting']):
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    # assign relevant part of JSON to venues
    venues = results['response']['venues']
    # tranform venues into a dataframe
    dataframe = json_normalize(venues)
    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]
    # filter the category for each row
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
    # clean column names by keeping only last term
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
    dataframe_filtered['Borough']= borough
    dataframe_filtered['Station Name']= station
    dataframe_filtered['Number exiting']= exits
    CoffeeShop.append(dataframe_filtered)

In [31]:
CoffeeShop_data = pd.concat(CoffeeShop, axis=0)
CoffeeShop_data.reset_index(drop=True, inplace=True)
CoffeeShop_data.drop(columns=['cc','country','crossStreet','formattedAddress','id','labeledLatLngs','state','neighborhood','city'], axis = 1,inplace=True)


### Let's see how many coffee shops have been returned for each tube station given the radius of 500 meters and the venue limit of 100:

In [32]:
print('Total number of Coffee Shops within 500 meters radius from a given station', len(CoffeeShop_data))
CoffeeShop_data.head(200)


Total number of Coffee Shops within 500 meters radius from a given station 1473


,Borough,Number exiting,Station Name,address,categories,distance,lat,lng,name,postalCode
0,City of London,16167,Aldgate,126 Whitechapel High St,Coffee Shop,210,51.514966,-0.072658,Costa Coffee,E1 7PU
1,City of London,16167,Aldgate,9 Aldgate High St,Coffee Shop,34,51.513990,-0.075459,Black Sheep Coffee,EC3N 1AH
2,City of London,16167,Aldgate,90 Mansell St,Coffee Shop,413,51.511502,-0.071572,Costa Coffee,E1 8AL
3,City of London,16167,Aldgate,83 Whitechapel High St,Coffee Shop,402,51.515919,-0.070309,Exmouth Coffee,E1 7QX
4,City of London,16167,Aldgate,2 Leman St,Coffee Shop,248,51.515066,-0.072126,Black Sheep Coffee,E1 8FA
5,City of London,16167,Aldgate,"133, Whitechapel High Street",Coffee Shop,157,51.514746,-0.073347,Department of Coffee & Social Affairs,NaN
6,City of London,16167,Aldgate,122 Leadenhall St,Coffee Shop,461,51.513736,-0.082098,Black Sheep Coffee,EC3V 4AB
7,City of London,16167,Aldgate,30 St Mary Axe,Coffee Shop,360,51.514643,-0.080671,Notes Coffee Roaster & Wine Bar,EC3A 8EP
8,City of London,16167,Aldgate,NaN,Coffee Shop,320,51.512188,-0.078651,Coffee Society,EC3M 4BR
9,City of London,16167,Aldgate,NaN,Coffee Shop,305,51.514543,-0.071105,Hyde Independent Specialty Coffee Bar,E1 8EN


In [33]:
map_coffee = folium.Map(location=[latitude, longitude], zoom_start=15)
# add markers to map
for lat, lng, borough, neighborhood in zip(stationsDataZone_1_Total['Latitude'], stationsDataZone_1_Total['Longitude'], stationsDataZone_1_Total['Borough'], stationsDataZone_1_Total['Station Name']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_coffee)    
# add popular spots to the map as blue circle markers
for lat, lng in zip(CoffeeShop_data.lat, CoffeeShop_data.lng):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5, 
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
        ).add_to(map_coffee)

# display map
map_coffee

### Analysis of coffee shops distribution by Boroughs and tube stations

In [34]:
CoffeeShopXBorough = CoffeeShop_data.groupby(['Borough']).size().reset_index(name='CountCoffeeShops Borough')
CoffeeShopXBorough.sort_values(by='CountCoffeeShops Borough', ascending=False)

,Borough,CountCoffeeShops Borough
8,Westminster,579
1,City of London,353
0,Camden,258
7,Tower Hamlets,67
6,Southwark,58
3,Islington,46
5,Lambeth,46
4,Kensington & Chelsea,38
2,Hackney,28


In [35]:
CoffeeShopXStation = CoffeeShop_data.groupby(['Borough','Station Name']).size().reset_index(name='CountCoffeeShops Station')
CoffeeShopXStation = CoffeeShopXStation.merge(CoffeeShopXBorough, left_on='Borough', right_on='Borough',suffixes=('_left', '_right'))
#CoffeeShopXStation.sort_values(by='CountCoffeeShops Borough', ascending=False).head(200)
stationsListToOptimize = stationsDataZone_1_Total.loc[stationsDataZone_1_Total['ID']!=166]
stationsListToOptimize.reset_index(drop=True, inplace=True)
stationsListToOptimize = stationsListToOptimize.merge(CoffeeShopXStation, left_on='Station Name', right_on='Station Name',suffixes=('_left', '_right'))
stationsListToOptimize['TrafficIndex'] = stationsListToOptimize['Number exiting'].div(stationsListToOptimize['CountCoffeeShops Station'],axis=0)
#stationsListToOptimize.sort_values(by='TrafficIndex', ascending=False).head(200)
print('The median traffic index for existing coffee shops in Zone 1 is:', stationsListToOptimize['TrafficIndex'].median())
stationsListFinal = stationsListToOptimize.loc[stationsListToOptimize['TrafficIndex']>stationsListToOptimize['TrafficIndex'].median()]
stationsListFinal.reset_index(drop=True, inplace=True)
stationsListFinal.rename(columns={"Borough_right": "Borough"}, inplace=True)
stationsListFinal.drop(columns=['Borough_left','Time Period'], axis = 1,inplace=True)
print('The final number of stations selected in Zone 1 is:', len(stationsListFinal))
stationsListFinal.sort_values(by='TrafficIndex', ascending=False).head(200)

The median traffic index for existing coffee shops in Zone 1 is: 1362.5059523809523
The final number of stations selected in Zone 1 is: 29


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ID,Latitude,Longitude,Station Name,Station Number,Zone,Number exiting,Borough,CountCoffeeShops Station,CountCoffeeShops Borough,TrafficIndex
25,273,51.4965,-0.1447,Victoria,741.0,1.0,127176,Westminster,19,579,6693.473684
23,236,51.4941,-0.1738,South Kensington,708.0,1.0,50662,Kensington & Chelsea,9,38,5629.111111
26,279,51.5036,-0.1143,Waterloo,747.0,1.0,147683,Lambeth,27,46,5469.740741
7,107,51.5067,-0.1428,Green Park,590.0,1.0,69611,Westminster,13,579,5354.692308
10,145,51.5308,-0.1238,King's Cross St. Pancras,625.0,1.0,147949,Camden,29,258,5101.689655
22,229,51.4924,-0.1565,Sloane Square,702.0,1.0,27657,Kensington & Chelsea,6,38,4609.500000
27,285,51.5010,-0.1254,Westminster,761.0,1.0,40204,Westminster,9,579,4467.111111
20,198,51.4893,-0.1334,Pimlico,776.0,1.0,17389,Westminster,4,579,4347.250000
18,193,51.5154,-0.1755,Paddington,670.0,1.0,81691,Westminster,19,579,4299.526316
14,157,51.5052,-0.0864,London Bridge,635.0,1.0,108319,Southwark,26,58,4166.115385


In [36]:
selectedStations = stationsListFinal
print('Number of stations selected by potential traffic :', len(stationsListFinal))
selectedStations.head(29)

Number of stations selected by potential traffic : 29


,ID,Latitude,Longitude,Station Name,Station Number,Zone,Number exiting,Borough,CountCoffeeShops Station,CountCoffeeShops Borough,TrafficIndex
0,7,51.5322,-0.1058,Angel,507.0,1.0,30631,Islington,14,46,2187.928571
1,11,51.5226,-0.1571,Baker Street,511.0,1.0,45436,Westminster,19,579,2391.368421
2,13,51.5133,-0.0886,Bank / Monument,513.0,1.0,136749,City of London,88,353,1553.965909
3,28,51.5142,-0.1494,Bond Street,524.0,1.0,61940,Westminster,35,579,1769.714286
4,87,51.5074,-0.1223,Embankment,542.0,1.0,35515,Westminster,18,579,1973.055556
5,89,51.5282,-0.1337,Euston,574.0,1.0,64272,Camden,17,258,3780.705882
6,99,51.4945,-0.1829,Gloucester Road,583.0,1.0,20619,Kensington & Chelsea,6,38,3436.500000
7,107,51.5067,-0.1428,Green Park,590.0,1.0,69611,Westminster,13,579,5354.692308
8,122,51.5009,-0.1925,High Street Kensington,605.0,1.0,20350,Kensington & Chelsea,6,38,3391.666667
9,133,51.5027,-0.1527,Hyde Park Corner,614.0,1.0,8198,Westminster,5,579,1639.600000


# Use decision optimization

## Step 1: Import the docplex package¶

In [37]:
import sys
import docplex.mp

## Step 2: Model the data

The data for this problem is quite simple: it is composed of the list of tube station selected from the previous section and their geographical locations.


## Step 3: Prepare the data

We need to collect the list of identified tube station locations and keep their names, latitudes, and longitudes.

In [38]:
# Store longitude, latitude and street crossing name of each public library location.
class XPoint(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __str__(self):
        return "P(%g_%g)" % (self.x, self.y)

class NamedPoint(XPoint):
    def __init__(self, name, x, y):
        XPoint.__init__(self, x, y)
        self.name = name
    def __str__(self):
        return self.name

Define how to compute the earth distance between 2 points
To easily compute distance between 2 points, we use the Python package geopy

In [39]:
try:
    import geopy.distance
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install geopy 
    else:
        !pip install --user geopy

In [40]:
# Simple distance computation between 2 locations.
from geopy.distance import great_circle
 
def get_distance(p1, p2):
    return great_circle((p1.y, p1.x), (p2.y, p2.x)).miles

In [41]:
listCoffeeLocation = []
for latitude, longitude, name in zip(selectedStations['Latitude'], selectedStations['Longitude'],selectedStations['Station Name']):
    cp = NamedPoint(name, longitude, latitude)
    listCoffeeLocation.append(cp)

In [42]:
print("There are %d suitable tube location in Zone1" % (len(listCoffeeLocation)))

There are 29 suitable tube location in Zone1


In [43]:
nb_shops = 5
print("We would like to open %d coffee shops" % nb_shops)

We would like to open 5 coffee shops


#### Validate the data by displaying them
We will use the folium library to display a map with markers.

In [44]:
try:
    import folium
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install folium 
    else:
        !pip install folium

In [45]:
import folium
map_osm = folium.Map(location=[latitude, longitude], zoom_start=13)
for station in listCoffeeLocation:
    lt = station.y
    lg = station.x
    folium.Marker([lt, lg]).add_to(map_osm)
map_osm

After running the above code, the data is displayed but it is impossible to determine where to ideally open the coffee shops by just looking at the map.

Let's set up DOcplex to write and solve an optimization model that will help us determine where to locate the coffee shops in an optimal way.

### Set up the prescriptive model

In [46]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Linux 64bit
* Python is present, version is 3.5.5
* docplex is present, version is (2, 8, 125)
* CPLEX wrapper is present, version is 12.8.0.0, located at: /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


Create the DOcplex model The model contains all the business constraints and defines the objective.

In [47]:
from docplex.mp.model import Model

mdl = Model("coffee shops")

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/docplex/mp/model.py:14: Warning: These editions of DOcplex and CPLEX for Watson Studio are in beta.
  warnings.warn('These editions of DOcplex and CPLEX for Watson Studio are in beta.', Warning)


Define the decision variables

In [48]:
BIGNUM = 999999999

# Ensure unique points
listCoffeeLocation = set(listCoffeeLocation)
# For simplicity, let's consider that coffee shops candidate locations are the same as station locations.
# That is: any station location can also be selected as a coffee shop.
coffeeshop_locations = listCoffeeLocation

# Decision vars
# Binary vars indicating which coffee shop locations will be actually selected
coffeeshop_vars = mdl.binary_var_dict(coffeeshop_locations, name="is_coffeeshop")
#
# Binary vars representing the "assigned" libraries for each coffee shop
link_vars = mdl.binary_var_matrix(coffeeshop_locations, listCoffeeLocation, "link")

Express the business constraints First constraint: if the distance is suspect, it needs to be excluded from the problem.

In [49]:
for c_loc in coffeeshop_locations:
    for b in listCoffeeLocation:
        if get_distance(c_loc, b) >= BIGNUM:
            mdl.add_constraint(link_vars[c_loc, b] == 0, "ct_forbid_{0!s}_{1!s}".format(c_loc, b))

Second constraint: each station must be linked to a coffee shop that is open.

In [50]:
mdl.add_constraints(link_vars[c_loc, b] <= coffeeshop_vars[c_loc]
                   for b in listCoffeeLocation
                   for c_loc in coffeeshop_locations)
mdl.print_information()

Model: coffee shops
 - number of variables: 870
   - binary=870, integer=0, continuous=0
 - number of constraints: 841
   - linear=841
 - parameters: defaults


Third constraint: each station is linked to exactly one coffee shop.

In [51]:
mdl.add_constraints(mdl.sum(link_vars[c_loc, b] for c_loc in coffeeshop_locations) == 1
                   for b in listCoffeeLocation)
mdl.print_information()

Model: coffee shops
 - number of variables: 870
   - binary=870, integer=0, continuous=0
 - number of constraints: 870
   - linear=870
 - parameters: defaults


Fourth constraint: there is a fixed number of coffee shops to open.

In [52]:
# Total nb of open coffee shops
mdl.add_constraint(mdl.sum(coffeeshop_vars[c_loc] for c_loc in coffeeshop_locations) == nb_shops)

# Print model information
mdl.print_information()

Model: coffee shops
 - number of variables: 870
   - binary=870, integer=0, continuous=0
 - number of constraints: 871
   - linear=871
 - parameters: defaults


Express the objective The objective is to minimize the total distance from stations to coffee shops so that a commuter always gets to our coffee shop easily.

In [53]:
# Minimize total distance from points to hubs
total_distance = mdl.sum(link_vars[c_loc, b] * get_distance(c_loc, b) for c_loc in coffeeshop_locations for b in listCoffeeLocation)
mdl.minimize(total_distance)

Solve with the Decision Optimization solve service Solve the model.

In [54]:
print("# coffee shops locations = %d" % len(coffeeshop_locations))
print("# coffee shops           = %d" % nb_shops)

assert mdl.solve(), "!!! Solve of the model fails"

# coffee shops locations = 29
# coffee shops           = 5


### Investigate the solution and then run an example analysis

The solution can be analyzed by displaying the location of the coffee shops on a map.

In [55]:
total_distance = mdl.objective_value
open_coffeeshops = [c_loc for c_loc in coffeeshop_locations if coffeeshop_vars[c_loc].solution_value == 1]
not_coffeeshops = [c_loc for c_loc in coffeeshop_locations if c_loc not in open_coffeeshops]
edges = [(c_loc, b) for b in listCoffeeLocation for c_loc in coffeeshop_locations if int(link_vars[c_loc, b]) == 1]

print("Total distance = %g" % total_distance)
print("# coffee shops  = {0}".format(len(open_coffeeshops)))
for c in open_coffeeshops:
    print("new coffee shop: {0!s}".format(c))

Total distance = 16.9946
# coffee shops  = 5
new coffee shop: Hyde Park Corner
new coffee shop: Lancaster Gate
new coffee shop: Westminster
new coffee shop: King's Cross St. Pancras
new coffee shop: Bank / Monument


Displaying the solution Coffee shops are highlighted in red.

In [56]:
import folium
map_osm = folium.Map(location=[51.5098, -0.1342], zoom_start=13)
for coffeeshop in open_coffeeshops:
    lt = coffeeshop.y
    lg = coffeeshop.x
    folium.Marker([lt, lg], icon=folium.Icon(color='red',icon='info-sign')).add_to(map_osm)
    
for b in listCoffeeLocation:
    if b not in open_coffeeshops:
        lt = b.y
        lg = b.x
        folium.Marker([lt, lg]).add_to(map_osm)
    

for (c, b) in edges:
    coordinates = [[c.y, c.x], [b.y, b.x]]
    map_osm.add_child(folium.PolyLine(coordinates, color='#FF0000', weight=5))

map_osm

# END